In [8]:

import pandas as pd
from ydata_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
#from lazypredict.Supervised import LazyRegressor


In [9]:
data = pd.read_csv("train.csv")
# tracking_id
# datetime
# wind_speed(m/s)
# atmospheric_temperature(C)
# shaft_temperature(C)
# blades_angle
# gearbox_temperature(C)
# engine_temperature(C)
# motor_torque(N-m)
# generator_temperature(C)
# atmospheric_pressure(Pascal)
# area_temperature(C)
# windmill_body_temperature(C)
# wind_direction
# resistance(ohm)
# rotor_torque(N-m)
# turbine_status
# cloud_level
# blade_length(m)
# blade_breadth(m)
# windmill_height(m)
# windmill_generated_power(kW/h)

#stat = data.describe()
data = data.drop("tracking_id",axis=1)
data = data.drop("datetime",axis=1)
data = data.dropna()
# report = ProfileReport(data, title = "report", explorative=True)
# report.to_file("report.html")

for col in data.columns:
  print(col)

# target = "windmill_generated_power(kW/h)"
# x = data.drop(target,axis=1)
# y = data[target]
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2024)

wind_speed(m/s) 
atmospheric_temperature(C)
shaft_temperature(C)
blades_angle
gearbox_temperature(C)
engine_temperature(C)
motor_torque(N-m)
generator_temperature(C)
atmospheric_pressure(Pascal)
area_temperature(C)
windmill_body_temperature(C)
wind_direction
resistance(ohm)
rotor_torque(N-m)
turbine_status
cloud_level
blade_length(m)
blade_breadth(m)
windmill_height(m)
windmill_generated_power(kW/h)
